In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_validate
import numpy as np
import random

In [2]:
data_path = 'Datasets/fe_scaled_one_hot_full_2.csv'

In [3]:
df_fe_scaled_one_hot_full = pd.read_csv(data_path)

In [28]:
df_fe_scaled_one_hot_full['STAT_CAUSE_DESCR_Structure'].sum()

3796

In [4]:
columns = df_fe_scaled_one_hot_full.columns

In [14]:
columns_set1 = ['FIRE_YEAR','DISCOVERY_DATE','DISCOVERY_DOY','DISCOVERY_TIME','LATITUDE','LONGITUDE',
                'SOURCE_SYSTEM_TYPE_FED','SOURCE_SYSTEM_TYPE_INTERAGCY','SOURCE_SYSTEM_TYPE_NONFED',
                'NWCG_REPORTING_AGENCY_BIA','NWCG_REPORTING_AGENCY_BLM','NWCG_REPORTING_AGENCY_BOR',
                'NWCG_REPORTING_AGENCY_DOD','NWCG_REPORTING_AGENCY_DOE','NWCG_REPORTING_AGENCY_FS',
                'NWCG_REPORTING_AGENCY_FWS','NWCG_REPORTING_AGENCY_IA','NWCG_REPORTING_AGENCY_NPS',
                'NWCG_REPORTING_AGENCY_ST/C&L','NWCG_REPORTING_AGENCY_TRIBE','FIRE_SIZE_CLASS_A','FIRE_SIZE_CLASS_B',
                'FIRE_SIZE_CLASS_C','FIRE_SIZE_CLASS_D','FIRE_SIZE_CLASS_E','FIRE_SIZE_CLASS_F','FIRE_SIZE_CLASS_G',
                'OWNER_DESCR_BIA','OWNER_DESCR_BLM','OWNER_DESCR_BOR','OWNER_DESCR_COUNTY','OWNER_DESCR_FOREIGN',
                'OWNER_DESCR_FWS','OWNER_DESCR_MISSING/NOT SPECIFIED','OWNER_DESCR_MUNICIPAL/LOCAL','OWNER_DESCR_NPS',
                'OWNER_DESCR_OTHER FEDERAL','OWNER_DESCR_PRIVATE','OWNER_DESCR_STATE','OWNER_DESCR_STATE OR PRIVATE',
                'OWNER_DESCR_TRIBAL','OWNER_DESCR_UNDEFINED FEDERAL','OWNER_DESCR_USFS','STATE_AK', 'STATE_AL', 
                'STATE_AR', 'STATE_AZ', 'STATE_CA', 'STATE_CO', 'STATE_CT', 'STATE_DC', 'STATE_DE', 'STATE_FL', 
                'STATE_GA', 'STATE_HI', 'STATE_IA', 'STATE_ID', 'STATE_IL', 'STATE_IN', 'STATE_KS', 'STATE_KY', 
                'STATE_LA', 'STATE_MA', 'STATE_MD', 'STATE_ME', 'STATE_MI', 'STATE_MN', 'STATE_MO', 'STATE_MS', 
                'STATE_MT', 'STATE_NC', 'STATE_ND', 'STATE_NE', 'STATE_NH', 'STATE_NJ', 'STATE_NM', 'STATE_NV', 
                'STATE_NY', 'STATE_OH', 'STATE_OK', 'STATE_OR', 'STATE_PA', 'STATE_PR', 'STATE_RI', 'STATE_SC', 
                'STATE_SD', 'STATE_TN', 'STATE_TX', 'STATE_UT', 'STATE_VA', 'STATE_VT', 'STATE_WA', 'STATE_WI', 
                'STATE_WV', 'STATE_WY']
# DISCOVERY_TIME,Discovery_DOY,LATITUDE,LONGITUTE, all state columns, 

columns_set2 = ['FIRE_YEAR','DISCOVERY_DATE','lat_lon_dist','lat_lon_angle',
                'SOURCE_SYSTEM_TYPE_FED','SOURCE_SYSTEM_TYPE_INTERAGCY','SOURCE_SYSTEM_TYPE_NONFED',
                'NWCG_REPORTING_AGENCY_BIA','NWCG_REPORTING_AGENCY_BLM','NWCG_REPORTING_AGENCY_BOR',
                'NWCG_REPORTING_AGENCY_DOD','NWCG_REPORTING_AGENCY_DOE','NWCG_REPORTING_AGENCY_FS',
                'NWCG_REPORTING_AGENCY_FWS','NWCG_REPORTING_AGENCY_IA','NWCG_REPORTING_AGENCY_NPS',
                'NWCG_REPORTING_AGENCY_ST/C&L','NWCG_REPORTING_AGENCY_TRIBE','FIRE_SIZE_CLASS_A','FIRE_SIZE_CLASS_B',
                'FIRE_SIZE_CLASS_C','FIRE_SIZE_CLASS_D','FIRE_SIZE_CLASS_E','FIRE_SIZE_CLASS_F','FIRE_SIZE_CLASS_G',
                'OWNER_DESCR_BIA','OWNER_DESCR_BLM','OWNER_DESCR_BOR','OWNER_DESCR_COUNTY','OWNER_DESCR_FOREIGN',
                'OWNER_DESCR_FWS','OWNER_DESCR_MISSING/NOT SPECIFIED','OWNER_DESCR_MUNICIPAL/LOCAL','OWNER_DESCR_NPS',
                'OWNER_DESCR_OTHER FEDERAL','OWNER_DESCR_PRIVATE','OWNER_DESCR_STATE','OWNER_DESCR_STATE OR PRIVATE',
                'OWNER_DESCR_TRIBAL','OWNER_DESCR_UNDEFINED FEDERAL','OWNER_DESCR_USFS','DISCOVERY_TIME_BIN_A', 
                'DISCOVERY_TIME_BIN_E', 'DISCOVERY_TIME_BIN_EM','DISCOVERY_TIME_BIN_LN', 'DISCOVERY_TIME_BIN_M',
                'DISCOVERY_TIME_BIN_N','DISCOVERY_DOY_BIN_FALL','DISCOVERY_DOY_BIN_SPRING','DISCOVERY_DOY_BIN_SUMMER',
                'DISCOVERY_DOY_BIN_WINTER', 'REGION_East','REGION_Mid_West', 'REGION_Mountain_West',
                'REGION_North_East','REGION_North_West', 'REGION_Pacific_West', 'REGION_South','REGION_South_East']

In [15]:
len(columns_set1)

95

In [16]:
len(columns_set2)

59

In [6]:
for column in df_fe_scaled_one_hot_full.columns:
  if df_fe_scaled_one_hot_full[column].isna().sum()!=0:
    print(column) #no column with na values...good!

In [138]:
#do not need below lines because we are choosing columns according to the requirements from above cell
# df_dropped_state = df_fe_scaled_one_hot_full.copy()
# state_columns = [col for col in df_dropped_state.columns if 'state_' in col.lower()]
# df_dropped_state = df_dropped_state.drop(columns = state_columns)

In [5]:
columns_to_predict = []
for column in df_fe_scaled_one_hot_full.columns:
  if 'STAT_CAUSE_DESCR' in str(column):
    columns_to_predict.append(column)
print(columns_to_predict)

['STAT_CAUSE_DESCR_Arson', 'STAT_CAUSE_DESCR_Campfire', 'STAT_CAUSE_DESCR_Children', 'STAT_CAUSE_DESCR_Debris Burning', 'STAT_CAUSE_DESCR_Equipment Use', 'STAT_CAUSE_DESCR_Fireworks', 'STAT_CAUSE_DESCR_Lightning', 'STAT_CAUSE_DESCR_Miscellaneous', 'STAT_CAUSE_DESCR_Missing/Undefined', 'STAT_CAUSE_DESCR_Powerline', 'STAT_CAUSE_DESCR_Railroad', 'STAT_CAUSE_DESCR_Smoking', 'STAT_CAUSE_DESCR_Structure']


In [6]:
df_fe_scaled_one_hot_full['STAT_CAUSE_DESCR'] = 'None'
i=1
for column in columns_to_predict:
  df_fe_scaled_one_hot_full.loc[df_fe_scaled_one_hot_full[column] == 1,'STAT_CAUSE_DESCR']= i
  i+=1

In [7]:
df_fe_scaled_one_hot_full.head()

,Unnamed: 0,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,LATITUDE,LONGITUDE,lat_lon_dist,lat_lon_angle,SOURCE_SYSTEM_TYPE_FED,...,DISCOVERY_DOY_BIN_WINTER,REGION_East,REGION_Mid_West,REGION_Mountain_West,REGION_North_East,REGION_North_West,REGION_Pacific_West,REGION_South,REGION_South_East,STAT_CAUSE_DESCR
0,0,0.193607,0.139590,-1.462914,-0.517416,0.530333,-1.513488,1.507126,-1.222123,1,...,1,0,0,0,0,0,1,0,0,8
1,1,0.043527,0.030331,-0.352283,-2.056039,0.350518,-1.477513,1.470465,-1.223782,1,...,0,0,0,0,0,0,1,0,0,7
2,2,0.043527,0.038135,-0.141263,1.582551,0.358844,-1.497320,1.490275,-1.223709,1,...,0,0,0,0,0,0,1,0,0,4
3,3,0.043527,0.049636,0.169714,0.497061,0.289615,-1.448135,1.440951,-1.224343,1,...,0,0,0,0,0,0,1,0,0,7
4,4,0.043527,0.049636,0.169714,0.497061,0.289615,-1.449315,1.442129,-1.224343,1,...,0,0,0,0,0,0,1,0,0,7


In [8]:
X = df_fe_scaled_one_hot_full[columns_set1]
# X2 = df_fe_scaled_one_hot_full[columns_set2]
Y = df_fe_scaled_one_hot_full['STAT_CAUSE_DESCR']
# Y = Y.astype('int')

In [24]:
x_train,x_valid,y_train,y_valid = train_test_split(X,Y,test_size = 0.2)
# x_train2,x_valid2,y_train,y_valid = train_test_split(X2,Y,test_size = 0.2)

In [25]:
x_train = x_train.to_numpy()
# x_train2 = x_train2.to_numpy()
y_train = y_train.to_numpy()
y_train = y_train.astype('int')
x_valid = x_valid.to_numpy()
# x_valid2 = x_valid2.to_numpy()
y_valid = y_valid.to_numpy()
y_valid = y_valid.astype('int')

First, check the baselines once again with the normal data

In [27]:
histgb = HistGradientBoostingClassifier().fit(x_train, y_train)
y_pred = histgb.predict(x_valid)
accuracy = accuracy_score(y_valid,y_pred)
accuracy

0.5413368501939679

In [26]:
RFC = RandomForestClassifier(max_depth=100, random_state=0,bootstrap=True,n_jobs=-1).fit(x_train, y_train)
y_pred = RFC.predict(x_valid)
accuracy = accuracy_score(y_valid,y_pred)
accuracy

0.597679297407822

In [28]:
MLP_classifier = MLPClassifier(solver = 'lbfgs',alpha = 1e-2, hidden_layer_sizes = (64,32,32),
                               random_state = 1,verbose=False, max_iter=200,activation = 'relu')
MLP_classifier.fit(x_train,y_train)
y_pred= MLP_classifier.predict(x_valid)
accuracy = accuracy_score(y_valid,y_pred)
accuracy


/Users/manikantamandlem/miniconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.5313738889051378

Now checking how the baselines look like with feature engineered data

In [17]:
histgb = HistGradientBoostingClassifier().fit(x_train2, y_train)
y_pred = histgb.predict(x_valid2)
accuracy = accuracy_score(y_valid,y_pred)
accuracy

0.5612175711858504

In [16]:
RFC = RandomForestClassifier(max_depth=100, random_state=0,bootstrap=True,n_jobs=-1).fit(x_train2, y_train)
y_pred = RFC.predict(x_valid2)
accuracy = accuracy_score(y_valid,y_pred)
accuracy

0.5805532142315863

In [ ]:
# We will not be using MLP with feature engineering data by assuming that feature engineering is not needed for MLP
# MLP_classifier = MLPClassifier(solver = 'lbfgs',alpha = 1e-2, hidden_layer_sizes = (64,32,32),
#                                random_state = 1,verbose=False, max_iter=200,activation = 'relu')
# MLP_classifier.fit(x_train2,y_train)
# y_pred= MLP_classifier.predict(x_valid)
# accuracy = accuracy_score(y_valid,y_pred)
# accuracy

Now we will do grid search cv to find best parameters for each of the below classifiers
First, grid search cv for histGradientBoostClassifier
Second, RandomForestClassifier
Third, Neural Network
Fourth, ExtraTreesClassifier
Fifth, Decision Tree CV with VotingClassifier

In [34]:
# parameters = {#'loss':['log_loss', 'auto', 'binary_crossentropy', 'categorical_crossentropy'],
#                 'loss':['categorical_crossentropy','auto']}
# #               'learning_rate':[0.001,0.005,0.01,0.05,0.1,0.2,0.3,0.4,0.5]},
# #                 'learning_rate':[0.2],
# #                 'max_leaf_nodes':[5,10,15,20,25,30,35,40,45,50]}
# #                 'max_leaf_nodes':[45],
# #               'max_iter':[100],
#               'max_depth':[5,10,15,20,25,30,35,40,50,60,70,None]}
# #                 'max_depth':[70],
# #               'min_samples_leaf':[5,10,15,20,25,30,35,40,50,60,80,100]}
# #                 'min_samples_leaf':[100]}
# #               'l2_regularization':[0]}
# parameters

{'loss': ['categorical_crossentropy', 'auto']}

In [35]:
# histgb = HistGradientBoostingClassifier()
# clf = GridSearchCV(histgb, parameters)
# histgb_results = clf.fit(x_train,y_train)

In [36]:
# histgb_result = pd.DataFrame.from_dict(histgb_results.cv_results_)
# histgb_result[histgb_result['rank_test_score']==1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_loss,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,39.15159,15.407636,0.784417,0.031106,categorical_crossentropy,{'loss': 'categorical_crossentropy'},0.228194,0.228165,0.228122,0.228125,0.228112,0.228144,0.000031,1


In [233]:
# histgb = HistGradientBoostingClassifier(loss='binary_crossentropy',learning_rate=0.2,max_leaf_nodes=45,
#                                         max_depth=70,min_samples_leaf=100)
# histgb.fit(x_train,y_train)
# y_pred = histgb.predict(x_valid)
# accuracy = accuracy_score(y_valid,y_pred)
# accuracy

0.8505821698356524

In [ ]:
#hist gradient boosting classifier on feature engineered data

In [28]:
# parameters = {#'loss':['log_loss', 'auto', 'binary_crossentropy', 'categorical_crossentropy'],
#                 'loss':['auto'],
#               'learning_rate':[0.1],#0.556
# #                 'learning_rate':[0.5],
#                 'max_leaf_nodes':[110,150,None]} #110 - 0.57
# #                 'max_leaf_nodes':[300],
# #               'max_iter':[100],
# #               'max_depth':[5,10,15,20,25,30,35,40,50,60,70,None]}
# #                 'max_depth':[100],
# #               'min_samples_leaf':[5,10,15,20,25,30,35,40,50,60,80,100]}
# #                 'min_samples_leaf':[120]}
# #               'l2_regularization':[0]}
# parameters

{'loss': ['auto'], 'learning_rate': [0.1], 'max_leaf_nodes': [110, 150, None]}

In [ ]:
# histgb = HistGradientBoostingClassifier()
# clf = GridSearchCV(histgb, parameters)
# histgb_results = clf.fit(x_train2,y_train)

In [ ]:
# histgb_result = pd.DataFrame.from_dict(histgb_results.cv_results_)
# histgb_result[histgb_result['rank_test_score']==1]

In [ ]:
# histgb = HistGradientBoostingClassifier(loss='binary_crossentropy',learning_rate=0.5,max_leaf_nodes=300,
#                                         max_depth=100,min_samples_leaf=120)
# histgb.fit(x_train2,y_train)
# y_pred = histgb.predict(x_valid2)
# accuracy = accuracy_score(y_valid,y_pred)
# accuracy

In [9]:
x_train,x_valid,y_train,y_valid = train_test_split(X,Y,test_size = 0.1)
# x_train2,x_valid2,y_train,y_valid = train_test_split(X2,Y,test_size = 0.1)

In [10]:
x_train = x_train.to_numpy()
# x_train2 = x_train2.to_numpy()
y_train = y_train.to_numpy()
y_train = y_train.astype('int')
x_valid = x_valid.to_numpy()
# x_valid2 = x_valid2.to_numpy()
y_valid = y_valid.to_numpy()
y_valid = y_valid.astype('int')

In [18]:
len(x_valid)

188047

In [ ]:
#RandomForestClassifier on normal data #baseline is 0.597

In [60]:
parameters = {
    'criterion':['gini'], #“gini”, “entropy”, “log_loss” 0.549..
    'max_depth' : [120], #0.55
    'bootstrap' : [True], #0.55
#     'max_depth':[20],
    'min_samples_split':[20,15],#0.56
    'min_samples_leaf':[1,5],#0.565
    'max_features':['sqrt', 'log2', None,0.7,0.8,0.9], #0.57
    'n_jobs':[-1]
#     ''
}

In [ ]:
RFC = RandomForestClassifier()
RFC_clf = GridSearchCV(RFC, parameters)
rfc_results = RFC_clf.fit(x_train,y_train)

In [62]:
rfc_result = pd.DataFrame.from_dict(rfc_results.cv_results_)
rfc_result[rfc_result['rank_test_score']==1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_jobs,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
13,24.598534,0.757016,0.329448,0.040468,True,gini,120,0.7,1,15,-1,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.575166,0.572124,0.575368,0.57114,0.575793,0.573918,0.001903,1


Now, testing on full dataset with validation data

In [11]:
RFC = RandomForestClassifier(criterion='gini',max_depth=150,bootstrap=True,min_samples_split=10,min_samples_leaf=1,
                            max_features=0.7,n_jobs=-1,random_state=0)

RFC.fit(x_train,y_train)

y_pred = RFC.predict(x_valid)
accuracy = accuracy_score(y_valid,y_pred)

In [12]:
accuracy

0.6188665599557558

In [13]:
f1score = f1_score(y_valid,y_pred,average='weighted')
f1score

0.6009399663615503

Below is with feature engineered data

In [33]:
parameters = {
    'criterion':['entropy'], #“gini”, “entropy”, “log_loss” 0.53
    'max_depth' : [30], #0.53
    'bootstrap' : [True], #0.53
    'min_samples_split':[10],#0.55
    'min_samples_leaf':[1], #0.55
    'max_features':[0.7],#0.57
    'n_jobs':[-1]
#     ''
}

In [34]:
RFC = RandomForestClassifier()
RFC_clf = GridSearchCV(RFC, parameters)
rfc_results = RFC_clf.fit(x_train2,y_train)

In [35]:
rfc_result = pd.DataFrame.from_dict(rfc_results.cv_results_)
rfc_result[rfc_result['rank_test_score']==1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_jobs,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,30.241315,0.518757,0.329206,0.022174,True,entropy,30,0.7,1,10,-1,"{'bootstrap': True, 'criterion': 'entropy', 'm...",0.566871,0.570209,0.571193,0.568933,0.562206,0.567882,0.003185,1


In [ ]:
RFC = RandomForestClassifier(criterion='entropy',max_depth=50,bootstrap=True,min_samples_split=5,min_samples_leaf=1,
                            max_features=0.7,n_jobs=-1,random_state=0)

RFC.fit(x_train2,y_train)

y_pred = RFC.predict(x_valid2)
accuracy = accuracy_score(y_valid,y_pred)


In [14]:
accuracy

0.615218535791584

In [15]:
f1score = f1_score(y_valid,y_pred,average='weighted')
f1score

0.600189853035171